In [22]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier

In [2]:
data = pd.read_csv('telecom_churn.csv')

In [4]:
data.head(3)

,Account length,Area code,International plan,Number vmail messages,Total day minutes,Total day calls,Total day charge,Total eve minutes,Total eve calls,Total eve charge,Total night minutes,Total night calls,Total night charge,Total intl minutes,Total intl calls,Total intl charge,Customer service calls,Churn
0,128,415,No,25,265.1,110,45.07,197.4,99,16.78,244.7,91,11.01,10.0,3,2.70,1,False
1,107,415,No,26,161.6,123,27.47,195.5,103,16.62,254.4,103,11.45,13.7,3,3.70,1,False
2,137,415,No,0,243.4,114,41.38,121.2,110,10.30,162.6,104,7.32,12.2,5,3.29,0,False


In [3]:
data.drop(['State', 'Voice mail plan'], axis=1, inplace=True)

In [5]:
data['International plan'] = data['International plan'].map({'No': 0, 'Yes': 1})

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3333 entries, 0 to 3332
Data columns (total 18 columns):
Account length            3333 non-null int64
Area code                 3333 non-null int64
International plan        3333 non-null int64
Number vmail messages     3333 non-null int64
Total day minutes         3333 non-null float64
Total day calls           3333 non-null int64
Total day charge          3333 non-null float64
Total eve minutes         3333 non-null float64
Total eve calls           3333 non-null int64
Total eve charge          3333 non-null float64
Total night minutes       3333 non-null float64
Total night calls         3333 non-null int64
Total night charge        3333 non-null float64
Total intl minutes        3333 non-null float64
Total intl calls          3333 non-null int64
Total intl charge         3333 non-null float64
Customer service calls    3333 non-null int64
Churn                     3333 non-null bool
dtypes: bool(1), float64(8), int64(9)
memory usage

In [7]:
y = data['Churn'].astype('int')

In [8]:
x = data.drop(['Churn'], axis=1)

In [10]:
x.shape, y.shape

((3333, 17), (3333,))

In [14]:
from sklearn.model_selection import train_test_split, cross_val_score

In [18]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=17)

In [19]:
x_train.shape, x_test.shape

((2333, 17), (1000, 17))

In [20]:
first_tree = DecisionTreeClassifier(random_state=17)

In [23]:
np.mean(cross_val_score(first_tree, x_train, y_train, cv=5))

0.9138423504976518

In [24]:
from sklearn.neighbors import KNeighborsClassifier

In [25]:
first_knn = KNeighborsClassifier()

In [26]:
np.mean(cross_val_score(first_knn, x_train, y_train, cv=5))

0.8671274043984523

In [27]:
from sklearn.model_selection import GridSearchCV

In [28]:
tree_params = {'max_depth': np.arange(1, 11), 'max_features': [0.5, 0.7, 1.0]}

In [29]:
tree_grid = GridSearchCV(first_tree, tree_params, cv=5, n_jobs=-1)

In [34]:
%%time
tree_grid.fit(x_train, y_train)

CPU times: user 226 ms, sys: 17 ms, total: 243 ms
Wall time: 1.62 s


In [35]:
tree_grid.best_params_

{'max_depth': 6, 'max_features': 1.0}

In [36]:
tree_grid.best_score_

0.9417059579939991

In [50]:
knn_params = {'n_neighbors': list(range(9, 30)) + list(range(50, 100, 10))}

In [51]:
knn_grid = GridSearchCV(first_knn, knn_params, cv=5, n_jobs=-1)

In [52]:
%%time
knn_grid.fit(x_train, y_train)

CPU times: user 683 ms, sys: 41 ms, total: 724 ms
Wall time: 20.9 s


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=5, p=2,
           weights='uniform'),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'n_neighbors': [9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 50, 60, 70, 80, 90]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [53]:
knn_grid.best_params_

{'n_neighbors': 9}

In [56]:
knn_grid.best_score_

0.8731247321045864

In [57]:
tree_grid.best_estimator_

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=6,
            max_features=1.0, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=17,
            splitter='best')

In [59]:
tree_preds = tree_grid.predict(x_test)

In [60]:
from sklearn.metrics import accuracy_score

In [61]:
accuracy_score(tree_preds, y_test)

0.946

In [62]:
1 - np.mean(y)

0.8550855085508551

In [63]:
from sklearn.tree import export_graphviz

In [64]:
export_graphviz(tree_grid.best_estimator_, out_file='telecom_tree.dot', feature_names=x.columns, filled=True)

In [65]:
!ls

mlcourse_ai_task_3.ipynb  telecom_churn.csv  telecom_tree.dot


In [66]:
!dot -Tpng telecom_tree.dot -o telecom_tree.png

/bin/sh: 1: dot: not found
